# Life Fairness Demo

End-to-end walkthrough: generate synthetic life underwriting data, fit GLM / NN / adversarial NN models to predict **one-year mortality risk**, and then derive **underwriting decline** decisions by flagging the top 2% highest-risk policies. We evaluate accuracy and fairness of these decline decisions across Race A/B (with territory and credit score proxy effects).

## 1. Setup

In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / "src").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Using project root: {PROJECT_ROOT}")

Using project root: C:\Users\hotfo\bias_mitigation_insurance_pricing


In [2]:
from __future__ import annotations

import numpy as np
import pandas as pd
import torch
from dataclasses import replace
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

from src.life.data_generation_life import (
    generate_life_underwriting_data,
    train_test_split_life,
)
from src.config import TrainingConfig
from src.evaluation.metrics import compute_accuracy_metrics
from src.evaluation.fairness import fairness_metrics
from src.models.glm_model import GLMClassifier
from src.models.nn_model import PlainNN, train_plain_nn, predict_proba_plain_nn
from src.models.adv_nn_model import AdvPredictor, train_adv_nn, predict_proba_adv_nn

## 2. Generate Life Data

We use the Life DGP to simulate **mortality risk** and then derive underwriting **decline** decisions:

- Features: Age, BMI, smoker S, chronic flag C, risky occupation R, observed credit Z, and territory T.
- Latent / audit-only: Race (A/B) and latent credit Z*.
- Target: one-year mortality Y in {0,1} (Y = 1 = death / adverse event).

The true risk rule depends on Age, BMI, S, C, R, and **latent** Z* (race-neutral). Models only observe Z (biased) and the proxy T. We then define underwriting decisions by declining the top 2% highest-risk policies based on each model's scores.

In [3]:
df = generate_life_underwriting_data(n_samples=100_000, seed=123)
print(f"Dataset size: {len(df):,}")
df.head()

Dataset size: 100,000


,Age,BMI,S,C,R,Z,T,Y,Race,Z_star,BMI_cat
0,36.723343,31.445601,0,1,1,572.628564,1,0,A,606.446351,2
1,41.116834,19.285569,1,0,0,788.088459,0,0,B,765.105469,0
2,55.681394,28.392383,1,0,0,681.141338,1,0,B,755.330667,1
3,40.537163,25.158097,0,0,0,669.692265,0,0,B,620.452243,1
4,66.953875,28.000590,0,0,0,605.146233,1,1,B,626.387414,1


### 2.1 Sanity Checks: Proxy T, Biased Z, and BMI-Race Correlation

We verify the designed channels:
- T is correlated with Race (proxy).
- Observed credit Z is systematically lower for Race A, even though Z* is race-neutral.
- Race A has a higher fraction of overweight/obese BMI than Race B.

In [4]:
# Territory by Race
print("E[T | Race]:")
print(df.groupby("Race")["T"].mean())

# Observed vs latent credit by Territory
print("\nE[Z | T] and E[Z_star | T]:")
print(df.groupby("T")[['Z', 'Z_star']].mean())

# Event rate by Race
print("\nPr(Y=1 | Race):")
print(df.groupby("Race")["Y"].mean())

# BMI distribution by Race (rough categories)
def bmi_category(bmi):
    if bmi < 25:
        return "normal"
    elif bmi < 30:
        return "overweight"
    return "obese"

df["BMI_cat_text"] = df["BMI"].apply(bmi_category)
print("\nBMI category distribution by Race:")
print(pd.crosstab(df["Race"], df["BMI_cat_text"], normalize="index").round(3))


E[T | Race]:
Race
A    0.602184
B    0.201595
Name: T, dtype: float64

E[Z | T] and E[Z_star | T]:
            Z      Z_star
T                        
0  700.415010  700.522393
1  659.671383  699.880857

Pr(Y=1 | Race):
Race
A    0.149174
B    0.147205
Name: Y, dtype: float64

BMI category distribution by Race:
BMI_cat_text  normal  obese  overweight
Race                                   
A              0.351  0.277       0.373
B              0.399  0.249       0.352


## 3. Train/Test Split

In [5]:
df_train, df_test = train_test_split_life(df, test_size=0.2, seed=123)
print(f"Train size: {len(df_train):,}, Test size: {len(df_test):,}")

y_train = df_train["Y"].to_numpy(dtype=np.int64)
y_test = df_test["Y"].to_numpy(dtype=np.int64)

# Protected attribute for fairness evaluation: Race A vs B
A_train = (df_train["Race"] == "A").astype(int).to_numpy(dtype=np.int64)
A_test = (df_test["Race"] == "A").astype(int).to_numpy(dtype=np.int64)

# Territory proxy for adversarial training
T_train = df_train["T"].to_numpy(dtype=np.int64)
T_test = df_test["T"].to_numpy(dtype=np.int64)

Train size: 80,000, Test size: 20,000


## 4. Features, Scaling, and DataLoaders

We follow the Life DGP spec:
- Input vector x = (A_g, B_g, S, C, R, Z, T), where A_g = (Age-45)/10 and B_g = (BMI-27)/5.
- Scale continuous features A_g, B_g, Z using training set statistics.
- Keep binaries S, C, R, T unscaled.

These feed GLM, NN, and the adversarial NN.

In [6]:
# Standardized age and BMI per DGP
for frame in (df_train, df_test):
    frame["A_g"] = (frame["Age"] - 45.0) / 10.0
    frame["B_g"] = (frame["BMI"] - 27.0) / 5.0

feature_cols_cont = ["A_g", "B_g", "Z"]
binary_cols = ["S", "C", "R", "T"]

scaler = StandardScaler()
X_train_cont = scaler.fit_transform(df_train[feature_cols_cont])
X_test_cont = scaler.transform(df_test[feature_cols_cont])

X_train = np.concatenate(
    [X_train_cont, df_train[binary_cols].to_numpy()],
    axis=1,
).astype(np.float32)
X_test = np.concatenate(
    [X_test_cont, df_test[binary_cols].to_numpy()],
    axis=1,
).astype(np.float32)

# For GLM we just use the same design matrix
X_train_glm = X_train
X_test_glm = X_test

train_cfg = TrainingConfig()

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=123, stratify=y_train
)

def build_loader(X, y, batch_size, shuffle=True):
    ds = TensorDataset(torch.from_numpy(X).float(), torch.from_numpy(y).float())
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)

train_loader = build_loader(X_tr, y_tr, train_cfg.batch_size, shuffle=True)
val_loader = build_loader(X_val, y_val, train_cfg.batch_size, shuffle=False)
test_loader = build_loader(X_test, y_test, train_cfg.batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train.shape, X_test.shape, device

((80000, 7), (20000, 7), device(type='cpu'))

## 5. GLM Baseline

We fit a logistic GLM on the life underwriting features to predict mortality risk:
- Inputs: (A_g, B_g, S, C, R, Z, T) after scaling.
- Target: Y = 1 = one-year mortality event.

We use the same `GLMClassifier` wrapper as in the auto demo.

In [7]:
glm = GLMClassifier().fit(X_train_glm, y_train)
y_proba_glm = glm.predict_proba(X_test_glm)
# If predict_proba returns 2 columns, take the positive class (event)
if y_proba_glm.ndim == 2 and y_proba_glm.shape[1] == 2:
    y_proba_glm = y_proba_glm[:, 1]
# y_proba_glm is predicted probability of Y=1 (mortality event)

## 6. Neural Network (Plain NN)

Two-layer feedforward network trained on the same features to predict mortality risk.

In [8]:
plain_nn = PlainNN(input_dim=X_train.shape[1]).to(device)
train_plain_nn(plain_nn, train_loader, val_loader, train_cfg, device)
y_proba_nn = predict_proba_plain_nn(plain_nn, X_test, device=device)
# y_proba_nn is predicted probability of Y=1 (mortality event)

## 7. Adversarial Neural Network (Proxy-Aware on T)

We train an adversarial NN that predicts mortality risk Y while an adversary predicts **territory T** from the hidden representation (with gradient reversal). This discourages reliance on T while auditing fairness on Race A/B.

In [9]:
# Start from the plain training config and increase lambda_adv for proxy control
train_cfg_adv = replace(train_cfg, lambda_adv=0.3)

# Adversary uses T (territory) as the protected attribute
adv_ds = TensorDataset(
    torch.from_numpy(X_train).float(),
    torch.from_numpy(y_train).float(),
    torch.from_numpy(T_train).long(),
)
adv_loader = DataLoader(adv_ds, batch_size=train_cfg.batch_size, shuffle=True)

adv_model = AdvPredictor(input_dim=X_train.shape[1]).to(device)
train_adv_nn(adv_model, adv_loader, train_cfg_adv, device=device)
y_proba_adv = predict_proba_adv_nn(adv_model, X_test, device=device)
# y_proba_adv is predicted probability of Y=1 (mortality event)

Epoch 0 [warmup]
Epoch 1 [warmup]
Epoch 2 [warmup]
Epoch 3 [warmup]
Epoch 4 [warmup]
Epoch 5 [adversarial]
Epoch 6 [adversarial]
Epoch 7 [adversarial]
Epoch 8 [adversarial]
Epoch 9 [adversarial]


## 8. Underwriting Decline Rule (Top 2% Highest Risk)

Models output risk scores p-hat = Pr(Y=1 | x). We implement an underwriting rule that declines the top 2% highest-risk policies:

- For each model, choose a threshold tau so that about 2% of test policies have p-hat >= tau.
- Define Decline = 1 if p-hat >= tau (decline), Decline = 0 otherwise (accept).

We then evaluate:
- Accuracy metrics on the risk scores (AUC, Brier, etc.).
- Fairness of the decline decisions across Race A/B using EO and DP.

In [ ]:
DECLINE_RATE = 0.02  # 2% of policies declined

def summarize_at_decline_rate(model_name, y_true, y_proba, A_true, decline_rate=DECLINE_RATE):
    '''
    Evaluate a model at a fixed decline rate: decline the top `decline_rate`
    fraction of policies by predicted risk.

    - y_true: event indicator (Y = 1 = mortality)
    - y_proba: predicted risk scores Pr(Y=1 | x)
    - A_true: Race indicator (1 = Race A, 0 = Race B)
    '''

    # Threshold so that ~decline_rate of policies are declined (high risk)
    threshold = float(np.quantile(y_proba, 1.0 - decline_rate))
    y_decline = (y_proba >= threshold).astype(int)

    # Accuracy metrics still on continuous risk scores
    acc = compute_accuracy_metrics(y_true, y_proba)

    # Fairness metrics: use decline decision as the positive class
    fair = fairness_metrics(y_true, y_proba, A_true, threshold=threshold)

    return {
        "model": model_name,
        "threshold": threshold,
        "decline_rate": float(y_decline.mean()),
        **acc,
        **fair,
    }

summary_decline = pd.DataFrame(
    [
        summarize_at_decline_rate("GLM", y_test, y_proba_glm, A_test),
        summarize_at_decline_rate("NN", y_test, y_proba_nn, A_test),
        summarize_at_decline_rate("ADV_NN", y_test, y_proba_adv, A_test),
    ]
)

summary_decline[
    [
        "model",
        "roc_auc",
        "brier",
        "log_loss",
        "eo_gap_tpr",
        "eo_gap_fpr",
        "dp_diff",
        "dp_ratio",
        "threshold",
        "decline_rate",
    ]
]

,model,roc_auc,brier,log_loss,eo_gap_tpr,eo_gap_fpr,dp_diff,dp_ratio,threshold,decline_rate
0,GLM,0.793520,0.104417,0.342898,0.041531,0.015380,0.018971,1.098083,0.237611,0.2
1,NN,0.791565,0.105994,0.346882,0.040037,0.008392,0.012794,1.065424,0.258021,0.2
2,ADV_NN,0.792142,0.104873,0.344313,0.040898,0.009798,0.014118,1.072362,0.234330,0.2


## 10. Summary

In [11]:
text = (
    "### Key Insights (Life Underwriting - Decline Decisions)"
    f"- At ~{100*DECLINE_RATE:.0f}% decline rate, GLM achieves ROC AUC "
    f"{summary_decline.loc[summary_decline.model=='GLM', 'roc_auc'].iat[0]:.3f} "
    f"with EO gap (TPR) "
    f"{summary_decline.loc[summary_decline.model=='GLM', 'eo_gap_tpr'].iat[0]:.3f} "
    f"and DP ratio "
    f"{summary_decline.loc[summary_decline.model=='GLM', 'dp_ratio'].iat[0]:.2f}."
    f"- The plain NN typically improves AUC "
    f"({summary_decline.loc[summary_decline.model=='NN', 'roc_auc'].iat[0]:.3f}) "
    f"but may widen Race A/B gaps in decline decisions."
    f"- The adversarial NN (proxy-aware on T) trades a bit of AUC "
    f"({summary_decline.loc[summary_decline.model=='ADV_NN', 'roc_auc'].iat[0]:.3f}) "
    f"for smaller EO/DP gaps in decline decisions, by discouraging use of "
    f"the territory proxy T in the hidden representation."
)
print(text)

### Key Insights (Life Underwriting - Decline Decisions)- At ~2% decline rate, GLM achieves ROC AUC 0.794 with EO gap (TPR) 0.021 and DP ratio 1.38.- The plain NN typically improves AUC (0.792) but may widen Race A/B gaps in decline decisions.- The adversarial NN (proxy-aware on T) trades a bit of AUC (0.792) for smaller EO/DP gaps in decline decisions, by discouraging use of the territory proxy T in the hidden representation.
